## Initialize the project 

In [1]:
import asyncio
from typing import Dict, List, Optional, Union, Callable
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.formatting_utils import colored
from typing_extensions import Annotated
import autogen

from teachability import Teachability
from concurrent.futures import ThreadPoolExecutor, as_completed

import arxiv

import requests
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.pdf import partition_pdf

import os
import shutil
import requests
import pickle

import nest_asyncio
nest_asyncio.apply()

version = "0.0.2"
ProjectID = "AI_security"
initiate_db = False


Project_dir = f"./{ProjectID}/{version}"
if not os.path.exists(Project_dir): os.makedirs(Project_dir)
output_dir = f'{Project_dir}/pdf_output'
if not os.path.exists(output_dir): os.makedirs(output_dir)

db_dir = f'{Project_dir}/memo-db/'
# check if db_dir exists, delete it if it does
if initiate_db:
    if os.path.exists(db_dir): shutil.rmtree(db_dir)

    # create a list of papers that have been read and saved it in a pickle file
    read_papers = []
    with open(f'{Project_dir}/read_papers.pkl', 'wb') as f:
        pickle.dump(read_papers, f)

    # create a list of abstract that have been read and saved it in a pickle file
    read_abstracts = []
    with open(f'{Project_dir}/read_abstracts.pkl', 'wb') as f:
        pickle.dump(read_abstracts, f)

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4-32k", "gpt-4", "gpt4", "gpt-35-turbo-16k", "gpt-4-0613", "gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613"]
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

# Configuration for the Language Model (LLM)
llm_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 120,
    "seed": 42,
}

# Configuration for the manager using the same config_list as llm_config
manager_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 60,
    "seed": 42,
}


# Termination message definition
termination_msg = (
    lambda x: isinstance(x, dict)
    and str(x.get("content", "")).upper() == "TERMINATE"
)

LLM models:  ['gpt-4', 'gpt-4-32k', 'gpt-4-0613', 'gpt-35-turbo', 'gpt-35-turbo-16k']


## teach agent for some skills

In [2]:
def create_teachable_groupchat(assitant_name, user_name, db_dir, config_list, verbosity=0):
    
    # Start by instantiating any agent that inherits from ConversableAgent.
    assistant = autogen.ConversableAgent(
        name=assitant_name,  # The name is flexible, but should not contain spaces to work in group chat.
        llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
    )

    # Instantiate the Teachability capability. Its parameters are all optional.
    teachability = Teachability(
        verbosity=verbosity,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
        reset_db=False,  
        path_to_db_dir=db_dir,
        recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
    )

    # Now add the Teachability capability to the agent.
    teachability.add_to_agent(assistant)

    user = autogen.UserProxyAgent(
        name=user_name,
        human_input_mode="NEVER",
        is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
        max_consecutive_auto_reply=0,
        code_execution_config={"use_docker": False},
    )

    return assistant, user

In [3]:
if initiate_db:
    prompt = "For each memorization task, initiate your process with 'MEMORIZE_ARTICLE: The following passage is extracted from an article, titled article_title [article_url]: \n\n' Delve into the passage to discern and assess its key insights. If the content presents noteworthy information, make a point to memorize these details. Conversely, if the passage does not offer significant insights, there's no need to commit it to memory. Upon choosing to memorize, finalize your notes by including both the article's title and its URL, employing the format '[source: article_title, article_url]' for efficient future access and verification."

    instract_assistant, instract_user = create_teachable_groupchat("instract_assistant", "instract_user", db_dir, config_list, verbosity=3)

    instract_user.initiate_chat(instract_assistant, silent=True, message=prompt)

## Define functions

In [11]:
def initiate_chat_with_paper_info(paper, teachable_agent, user, query_text):
    user.initiate_chat(teachable_agent,
                       silent=True,
                       message=f"The following article is one of the articles that I found for '{query_text}' topic: \n\n '{paper.title}' by {paper.authors} updated on {paper.updated}: {paper.pdf_url} \nsummary: {paper.summary} \n?")

def process_query(query_text, n_results, teachable_agent, user):
    """Function to process each query and initiate chats for each paper found."""
    sort_by = arxiv.SortCriterion.Relevance
    papers = arxiv.Search(query=query_text, max_results=n_results, sort_by=sort_by)

    # check if the abstract has been read before
    with open(f'{Project_dir}/read_abstracts.pkl', 'rb') as f:
        read_abstracts = pickle.load(f)

    papers = list(arxiv.Client().results(papers))
    papers = [paper for paper in papers if paper.pdf_url not in read_abstracts]

    # add papers to the read_papers list
    with open(f'{Project_dir}/read_abstracts.pkl', 'rb') as f:
        read_abstracts = pickle.load(f)
    read_abstracts.extend([paper.pdf_url for paper in papers])
    with open(f'{Project_dir}/read_abstracts.pkl', 'wb') as f:
        pickle.dump(read_abstracts, f)

    
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(initiate_chat_with_paper_info, paper, teachable_agent, user, query_text) for paper in papers]
        for future in as_completed(futures):
            future.result()

def arxiv_retriever(queries: Annotated[List[str], "The list of query texts to search for."], 
                    n_results: Annotated[int, "The number of results to retrieve for each query."] = 10,
                    ) -> str:
    
    # Create a TeachableAgent and UserProxyAgent to represent the researcher and the user, respectively.
    arxiver, arxiver_user = create_teachable_groupchat("arxiver", "arxiver_user", db_dir, config_list, verbosity=0)


    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_query, query_text, n_results, arxiver, arxiver_user) for query_text in queries]
        for future in as_completed(futures):
            future.result()

    # Instantiate a UserProxyAgent to represent the user. But in this notebook, all user input will be simulated.
    return f"Dear Researcher, Database updated with on the following topics: {', '.join(list(queries))}. Please go ahead with your task. Contact me for updates if needed. Your research Coordinator"


message = ["Large Language Models safety and reliability", "AI systems reliability mechanisms", "Methodologies for improving AI safety", "Recent advancements in AI system safety", "Latest research in AI reliability"]
# arxiv_retriever(message, n_results=10)

********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
    Location = ./AI_security/0.0.2/memo-db/uid_text_dict.pkl
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
********************************
* SYNC DISK MEM

'Dear Researcher, Database updated with on the following topics: Large Language Models safety and reliability, AI systems reliability mechanisms, Methodologies for improving AI safety, Recent advancements in AI system safety, Latest research in AI reliability. Please go ahead with your task. Contact me for updates if needed. Your research Coordinator'

In [5]:
def check_reasoning(reason, summary):
    
    # Start by instantiating any agent that inherits from ConversableAgent.
    assistant = autogen.AssistantAgent(
        name="reasoning_checker",  # The name is flexible, but should not contain spaces to work in group chat.
        llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
    )

    user = autogen.UserProxyAgent(
        name="user",
        human_input_mode="NEVER",
        is_termination_msg=termination_msg,
        max_consecutive_auto_reply=0,
        code_execution_config={"use_docker": False},
    )

    chat_hist = user.initiate_chat(assistant, silent=True, message=f"check if \"{reason} is a good reason is to read a paper with the following summary: {summary} /n/n answer only with 'yes' or 'no'")
    return chat_hist.chat_history[-1]['content']

def download_pdf(url, save_path):
    """Download a PDF from a given URL."""
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)

def initiate_chat_read_paper(text, article):
    paper_reader, reader_user = create_teachable_groupchat("paper_reader", "reader_user", db_dir, config_list, verbosity=0)
    reader_user.initiate_chat(paper_reader,
                       silent=True,
                       message=f"MEMORIZE_ARTICLE: The following passage is extracted from an article titled '{article}': \n\n {text}."
                    )
    
def chunk_pdf(url, title):
    
    print(f"Reading the article, '{title}'")
    pdf_filename = url.split('/')[-1]
    pdf_path = os.path.join(output_dir, pdf_filename)
    

    download_pdf(url, pdf_path)
    elements = partition_pdf(filename=pdf_path)
    chunked_elements = chunk_by_title(elements)

    # find checked_elemnt that includes "REFERENCES"
    for i, chunk in enumerate(chunked_elements):
        if 'REFERENCES' in chunk.text.upper():
            chunked_elements = chunked_elements[:i]
            break

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(initiate_chat_read_paper, chunk.text, title) for chunk in chunked_elements if len(chunk.text.split()) > 30]
        for future in as_completed(futures):
            future.result()


"""
This `get_pdf` function is designed to download a PDF from a given URL, extract its content, 
partition the content into chunks based on titles, and then initiate a chat to share and memorize 
each chunk of the article with a teachable agent and a user.
"""
def get_pdf(urls: Annotated[List[str], "The URL of the PDF to convert."],
            reasons: Annotated[List[str], "The reasons for reading the papers."]
            ) -> str:
    
    urls_list = []
    titles_list = []
    for url in urls:

        paper_id = url.split('/')[-1].replace('.pdf', '')
        search_by_id = arxiv.Search(id_list=[paper_id])
        paper = list(arxiv.Client().results(search_by_id))[0]
        title = paper.title
        updated = paper.updated
        summary = paper.summary
        title = f"{title} [{url}] updated {updated}"
        
        check_reason = check_reasoning(reasons[urls.index(url)], summary)
        if 'no' in check_reason.lower():
            print(f"The article, '{title}', does not meet the criteria for reading.")
            continue
        
        # add url to list of papers in pickle file if it doesn't exist
        with open(f'{Project_dir}/read_papers.pkl', 'rb') as f:
            read_papers = pickle.load(f)

        if url in read_papers: 
            print(f"The article, '{title}', has already been read and shared with you in your memory.")
            continue
        else:
            urls_list.append(url)
            titles_list.append(title)

        read_papers.append(url)
        with open(f'{Project_dir}/read_papers.pkl', 'wb') as f:
            pickle.dump(read_papers, f)


    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(chunk_pdf, url, title) for url, title in zip(urls_list, titles_list)]
        for future in as_completed(futures):
            future.result() 


    return f"The articles {', and '.join(titles_list)}  has been read and the content has been shared with you in your memory."

# Example usage
# # pdf_urls = ["https://arxiv.org/pdf/2302.09270v3.pdf", "https://arxiv.org/pdf/2401.06373v2.pdf"]
# get_pdf(urls= ["https://arxiv.org/pdf/2104.07650.pdf"],
#         reasons=["understand the general context of reliability and safety in large language models"])


## Define Agents

In [6]:
# System message constants for different roles
PI_prompt = """You are a Principal investigator (PI): You are the leader of the research team who asks the questions and gives task.
You MUST make sure that the research team is on the right track and the research is going in the right direction. 
You should check the work of the researcher, coordinator and Planner and provide feedback to improve the quality of the work or confirm the work.
reply 'TERMINATE' in the end when everything is done.
"""


COORDINATOR = """You are a Research coordinator: This is the person who coordinates the various aspects of the research project. 
you are equipped wih a tool that could help you to query for the arxiv api. 
You MUST rephrase research questions into a list of queries (at least 5) for the arxiv api that cover the key aspects of the research questions. 
"""

RESEARCHER = """You are a Researcher: This is the person who performs the research and writes the final report/article.
You MUST take to account the feedback from the critic to improve the quality of the work.
The final report/article should include references to the sources you used to perform the research and also for reader to refer for more information.
The references MUST be in the format of the arxiv url.
You are equipped with a memory tool that could help you to retrieve the information you need. 
You are equipped with a tool that could help you to get the content of the pdf file from the url. In case you have a useful pdf url in your memory, which you can use to get the content of the pdf file, you should use the tool to get the content of the pdf file.
You MUST verify the information you retrieve from the memory tool before using it in your work an make sure it is accurate and enough to answer the question.
If there isn't enough information for you to perform your task, you should asked the PI to provide you with the missing information from Arxiv. You should mention what is missing in your request start you sentence exactly with "ARXIV REQUEST:". 
You should not generate answers that don't use the sources provided in the context.
"""
SUB1 = """You are a Sub-investigator (Sub-I): This is the assistant to the PI, who helps with the tasks of the PI with a step wise research plan with sub-research topics.
you MUST help PI to ensure that the research team is on the right track and the research is going in the right direction. 
In your role, you have the autonomy to question the provided content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant.
Reply 'TERMINATE' in the end when everything is done.
"""

# If there isn't enough information below, you should reply exactly 'UPDATE CONTEXT'.
QNA_PROMPT = """Assistant helps the researchers with searching information from the arxiv API. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. Do not generate answers that don't use the sources below. 
In your role, you have the autonomy to question the provided content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant.
For tabular information return it as an html table. Do not return markdown format. If the question is not in English, answer in the language used in the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, for example [info1.txt]. Don't combine sources, list each source separately.
User's question is: {input_question}

Context is: {input_context}"""

PLANNER = """Planner. Suggest a plan. Revise the plan based on feedback from PI and critic, until PI approval.
The plan may involve a Research Coordinator to to rephrase research questions into key word queries for the arxiv api and a Researcher who could performs the research of paper's summaries. 
Explain the plan first. Be clear which step is performed by an Research Coordinator, and which step is performed by a Researcher.
"""


# Agent definitions
principalInvestigator = autogen.UserProxyAgent(
    name="PI",
    is_termination_msg=termination_msg,
    human_input_mode="TERMINATE",
    system_message=PI_prompt,
    llm_config=llm_config, 
    code_execution_config=False,
    description="Principal investigator (PI) is the leader of the research team who asks the questions and gives task."
)


planner = autogen.AssistantAgent(
    name="Planner",
    system_message=PLANNER,
    llm_config=llm_config,
    description="Planner suggests a plan and revises the plan based on feedback from PI and critic, until PI approval."
)

subInvestigator = autogen.AssistantAgent(
    name="Sub-I",
    is_termination_msg=termination_msg,
    system_message=SUB1,
    llm_config=llm_config,
    description="Sub-investigator (Sub-I) is the assistant to the PI, who helps with the tasks of the PI with a step wise research plan with sub-research topics."
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check the work of researcher, research coordinator and Planner and provide feedback to improve the quality of the work",
    llm_config=llm_config,
    description="Critic is responsible for double checking the work of researcher, research coordinator and Planner and provide feedback to improve the quality of the work"
)


researchCoordinator = autogen.AssistantAgent(
    name="ResearchCoordinator",
    is_termination_msg=termination_msg,
    system_message=COORDINATOR,  # COORDINATOR should be a predefined string variable
    llm_config=llm_config,
    description="Research coordinator is the person who rephrase research questions into key word queries for the arxiv api."
)

# create a UserProxyAgent instance named "user_proxy"
RC_proxy = autogen.UserProxyAgent(
    name="ResearchCoordinator_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "content" in x
    and x["content"] is not None
    and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "ResearchCoordinator",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    description="assist Research coordinator to query for the arxiv api."
)

researcher = autogen.AssistantAgent(
    name="Researcher",
    is_termination_msg=termination_msg,
    system_message=RESEARCHER, 
    llm_config=llm_config,
    description="Researcher is the person who performs the research of paper's summaries."
)


from chromadb.config import Settings

# db_dir = './teachability_db_AI_safety'
# settings = Settings(
#             anonymized_telemetry=False, allow_reset=True, is_persistent=True, persist_directory=db_dir
#         )

# Researcher_aid = RetrieveUserProxyAgent(
#     name="RAG_USER",
#     is_termination_msg=termination_msg,
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=3,
#     llm_config=llm_config, 
#     retrieve_config={
#         "task": "code",
#         "customized_prompt": QNA_PROMPT, 
#         "docs_path": None,
#         "model": config_list[0]["model"],
#         "client": chromadb.Client(settings),
#         "collection_name": "memos",
#         "get_or_create": True,
#     },
#     code_execution_config=False,  # we don't want to execute code in this case.
#     description="Assistant has extra content retrieval power and can provide team members with context in database.",
# )

# def chromadb_retriever(
#     message: Annotated[
#         str,
#         "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
#     ],
#     n_results: Annotated[int, "number of results. it should be at least 10."],
# ) -> str:
#     Researcher_aid.n_results = n_results  # Set the number of results to be retrieved.
#     # Check if we need to update the context.
#     update_context_case1, update_context_case2 = Researcher_aid._check_update_context(message)
#     if (update_context_case1 or update_context_case2) and Researcher_aid.update_context:
#         Researcher_aid.problem = message if not hasattr(Researcher_aid, "problem") else Researcher_aid.problem
#         _, ret_msg = Researcher_aid._generate_retrieve_user_reply(message)
#     else:
#         _context = {"problem": message, "n_results": n_results}
#         ret_msg = Researcher_aid.message_generator(Researcher_aid, None, _context)
#     return ret_msg if ret_msg else message

# message = "Overview of time series forecasting methods"
# chromadb_retriever(message, n_results=3)

# Instantiate the Teachability capability. Its parameters are all optional.
teachability = Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=False,
    path_to_db_dir=db_dir,
    recall_threshold=1.2,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
teachability.add_to_agent(researcher)


********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
    Location = ./AI_security/0.0.2/memo-db/uid_text_dict.pkl


## setup group chat

In [7]:
from autogen import Agent
from typing import List, Dict

def custom_speaker_selection_func(last_speaker: Agent, groupchat: autogen.GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.
    # principalInvestigator, planner, researchCoordinator, researcher, critic, RC_proxy, Researcher_aid

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """
    messages = groupchat.messages

    if len(messages) <= 1:
        # first, let the researchCoordinator retrieve relevant data populate db
        return researchCoordinator

    # if last_speaker is researchCoordinator:
    #     # if the last message is from researchCoordinator, let the planner to speak
    #     return planner
    
    if last_speaker is planner:
        # if the last message is from researchCoordinator, let the planner to speak
        return critic

    else:
        # default to auto speaker selection method
        return "auto"



In [8]:
def _reset_agents():
    principalInvestigator.reset()
    subInvestigator.reset()
    researchCoordinator.reset()
    researcher.reset()
    planner.reset()
    critic.reset()
    RC_proxy.reset()
    # Researcher_aid.reset()


def call_chat(PROBLEM, clear_history=True):
    _reset_agents()  # Resets the state of all the agents before starting the chat

    autogen.agentchat.register_function(
        arxiv_retriever,
        caller=researchCoordinator,
        executor=RC_proxy,
        name="arxiv_retriever",
        description="Retrieve content for question answering from arxiv."
    )

    autogen.agentchat.register_function(
        get_pdf,
        caller=researcher,
        executor=RC_proxy,
        name="get_pdf",
        description="retrieve the content of the pdf file from the url."
    )

    # Create the GroupChat manager instance.
    groupchat = autogen.GroupChat(
        agents=[principalInvestigator, planner, researchCoordinator, researcher, critic, RC_proxy],
        # agents=[principalInvestigator, planner, researcher, critic, Researcher_aid],
        messages=[],
        max_round=35,
        speaker_selection_method="auto",
        allow_repeat_speaker=False,
    )

    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=manager_config,
    )

    # Initialize the chat with the primary investigator as the proxy agent.
    principalInvestigator.initiate_chat(
        manager,
        message=PROBLEM,
        clear_history=clear_history
    )

    # Start chatting with the primary investigator acting as the user proxy agent.
    return principalInvestigator.chat_messages


## Strat chat

In [9]:
# Example usage:
# You MUST Collect enough information from the arxiv API and summerize the information in the blog post.
# PROBLEM = """Write blog post about the modelling of reliability and safety mechanisms in AI system. 
# The focus MUST be on Large Language Models.
# The blog post MUST be written in a way that is easy to understand for a non-technical audience.
# The blog post MUST be up to date and include the latest research in the field in your database.
# """

PROBLEM = """As a recognized authority on enhancing the reliability and safety of AI systems, you're invited to illuminate our AI community with your insights through a blog post titled "Reliability and Safety Mechanisms in AI Systems and the most recent advancement".

 Your expertise will guide our audience through the nuances of ensuring AI operates within safe and reliable parameters, with a special focus on Large Language Models (LLMs). Here's how to structure your invaluable contribution:

- **Core Theme:** Anchor your discussion around Large Language Models, highlighting their significance in the current AI landscape and why reliability and safety are paramount.

- **Innovative Progress:** Dive into the latest breakthroughs and methodologies [at least 3 methodologies] that have emerged in the domain of AI reliability and safety. Showcase [with reference to original paper] how these advancements are shaping the future of responsible AI development and implementation.

- **Accessible Insight:** While your post will be rich in information, ensure it's crafted in a manner that demystifies complex concepts for those outside the tech sphere. Your goal is to enlighten, not overwhelm.

- **Credible Sources:** You MUST Strengthen your narrative by integrating references to the research, studies, and sources that informed your insights. Additionally, provide these references for readers seeking to delve deeper into the subject.

- **Current Perspective:** Reflect the cutting-edge of the field by incorporating the most recent findings and research available in your database. Your post should serve as a timely resource for anyone looking to understand the state-of-the-art in AI safety and reliability mechanisms.

This blog post is an opportunity to not just share knowledge but to foster a deeper understanding and appreciation for the ongoing efforts to make AI systems more reliable and safe for everyone. Your contribution will undoubtedly be a beacon for those navigating the complexities of AI in our increasingly digital world.
You are equipped  with a function that could read a paper for you. If you need a missing info please update you knowledge base.
"""
messages = call_chat(PROBLEM, clear_history=True)
print(messages)

PI (to chat_manager):

As a recognized authority on enhancing the reliability and safety of AI systems, you're invited to illuminate our AI community with your insights through a blog post titled "Reliability and Safety Mechanisms in AI Systems and the most recent advancement".

 Your expertise will guide our audience through the nuances of ensuring AI operates within safe and reliable parameters, with a special focus on Large Language Models (LLMs). Here's how to structure your invaluable contribution:

- **Core Theme:** Anchor your discussion around Large Language Models, highlighting their significance in the current AI landscape and why reliability and safety are paramount.

- **Innovative Progress:** Dive into the latest breakthroughs and methodologies [at least 3 methodologies] that have emerged in the domain of AI reliability and safety. Showcase [with reference to original paper] how these advancements are shaping the future of responsible AI development and implementation.

- 

Add of existing embedding ID: 6087
Add of existing embedding ID: 6088
Add of existing embedding ID: 6178
Add of existing embedding ID: 6190



>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
ResearchCoordinator_proxy (to chat_manager):

ResearchCoordinator_proxy (to chat_manager):

***** Response from calling tool "call_H3ogcCPMOHriRxOF8e0mpWuA" *****
Dear Researcher, Database updated with on the following topics: reliability of Large Language Models, advancements in AI safety and reliability, methodologies for AI reliability, research on AI safety for Large Language Models, responsible AI development techniques. Please go ahead with your task. Contact me for updates if needed. Your research Coordinator
**********************************************************************

--------------------------------------------------------------------------------
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
Researcher (to chat_manager):

***** Suggested tool Call (call_FDj0RgpzuifpjCQuKbJq0gOD): arxiv_retriever *****
Arguments: 
{"queries": 